Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.954973
Minibatch accuracy: 12.5%
Validation accuracy: 10.1%
Minibatch loss at step 50: 2.172522
Minibatch accuracy: 25.0%
Validation accuracy: 41.5%
Minibatch loss at step 100: 1.299857
Minibatch accuracy: 50.0%
Validation accuracy: 69.1%
Minibatch loss at step 150: 0.864242
Minibatch accuracy: 75.0%
Validation accuracy: 72.6%
Minibatch loss at step 200: 0.503801
Minibatch accuracy: 81.2%
Validation accuracy: 77.0%
Minibatch loss at step 250: 0.952262
Minibatch accuracy: 68.8%
Validation accuracy: 75.6%
Minibatch loss at step 300: 0.854922
Minibatch accuracy: 62.5%
Validation accuracy: 78.5%
Minibatch loss at step 350: 0.516290
Minibatch accuracy: 81.2%
Validation accuracy: 79.7%
Minibatch loss at step 400: 0.427212
Minibatch accuracy: 81.2%
Validation accuracy: 80.1%
Minibatch loss at step 450: 0.612374
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 500: 0.585504
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [20]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
  
  # Input data
  with tf.name_scope('input'):
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels), name='x-input')
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='y-input')
    tf_valid_dataset = tf.constant(valid_dataset, name='valid')
    tf_test_dataset = tf.constant(test_dataset, name='test')
  
  # Variables
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model 
  def model(data):
    data = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], padding='SAME')
    data = tf.nn.relu(data + layer1_biases)
    data = tf.nn.max_pool(data, [1,2,2,1], [1,2,2,1], padding='SAME')
  
    # Same as above, with layer 2 instead of 1
    data = tf.nn.conv2d(data, layer2_weights, [1,1,1,1], padding='SAME')
    data = tf.nn.relu(data + layer2_biases)
    data = tf.nn.max_pool(data, [1,2,2,1], [1,2,2,1], padding='SAME')  
    
    # reshape/flatten the data to get 2d array
    shape = data.get_shape().as_list()
    data = tf.reshape(data, [shape[0], shape[1] * shape[2] * shape[3]])
    
    # Fully connected
    data = tf.nn.relu(tf.matmul(data, layer3_weights) + layer3_biases)
    return tf.matmul(data, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [21]:
log_dir = "/tmp/tensorflow/udacity/"
if tf.gfile.Exists(log_dir):
  tf.gfile.DeleteRecursively(log_dir)
tf.gfile.MakeDirs(log_dir)

writer = tf.summary.FileWriter(log_dir, graph=graph)

num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print ("initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0]-batch_size)
    batch_data = train_dataset[offset:(offset+batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
      

initialized
Minibatch loss at step 0: 3.372923
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.034013
Minibatch accuracy: 18.8%
Validation accuracy: 21.7%
Minibatch loss at step 100: 1.453258
Minibatch accuracy: 50.0%
Validation accuracy: 58.5%
Minibatch loss at step 150: 1.141589
Minibatch accuracy: 68.8%
Validation accuracy: 66.7%
Minibatch loss at step 200: 0.774132
Minibatch accuracy: 75.0%
Validation accuracy: 74.8%
Minibatch loss at step 250: 1.318335
Minibatch accuracy: 68.8%
Validation accuracy: 73.4%
Minibatch loss at step 300: 0.755207
Minibatch accuracy: 75.0%
Validation accuracy: 77.9%
Minibatch loss at step 350: 0.552991
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.536795
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.592329
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
Minibatch loss at step 500: 0.476821
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [110]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
  
  # Input data
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels), name='x-input')
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='y-input')
#   tf_valid_dataset = tf.constant(valid_dataset, name='valid-input')
#   tf_test_dataset = tf.constant(test_dataset, name='test-input')
  
  # We can't initialize these variables to 0 - the network will get stuck.
  def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

  def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

  def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      tf.summary.histogram('histogram', var)

  def nn_layer(input_tensor, dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses ReLU to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable(dim)
        variable_summaries(weights)
      with tf.name_scope('biases'):
        biases = bias_variable([dim[-1]])
        variable_summaries(biases)
      with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.summary.histogram('pre_activations', preactivate)
      activations = act(preactivate, name='activation')
      tf.summary.histogram('activations', activations)
      return activations    
    
  def conv2d_layer(input_tensor, dim, layer_name):
    """Reusable code for making a simple conv2d layer.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable(dim)
        variable_summaries(weights)
      with tf.name_scope('biases'):
        biases = bias_variable([dim[-1]])
        variable_summaries(biases)
      with tf.name_scope('conv2d'):
        preactivate = tf.nn.conv2d(input_tensor, weights, [1,1,1,1], padding='VALID')
        tf.summary.histogram('pre_activations', preactivate)
      activations = tf.nn.relu(preactivate, name='activation')
      tf.summary.histogram('activations', activations)
      return activations   
  

  def get_weight_and_bias(size):
    with tf.name_scope("weight"):
      weights = weight_variable(size)
      variable_summaries(weights)
    with tf.name_scope("bias"):
      bias = bias_variable([size[-1]])
      variable_summaries(bias)
#         layer1_biases = tf.Variable(tf.zeros([size[-1]]), name='layer1_b')
    return weights, bias
    
  # Variables

  data = tf_train_dataset
#   def model(data):
  # C1 = input image is 28x28
  with tf.name_scope("Conv_1"):
    layer1_weights, layer1_biases = get_weight_and_bias([patch_size, patch_size, num_channels, depth])
    data = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], padding='VALID')
    data = tf.nn.relu(data + layer1_biases)

  # S2 = Avg Pooling layer, input is 24x24
  with tf.name_scope("Pool_2"):
    data = tf.nn.avg_pool(data, [1,2,2,1], [1,2,2,1], padding='VALID')

  # C3 = input of 12x12
  with tf.name_scope("Conv_3"):
    layer2_weights, layer2_biases = get_weight_and_bias([patch_size, patch_size, depth, depth])
    data = tf.nn.conv2d(data, layer2_weights, [1,1,1,1], padding='VALID')
    data = tf.nn.relu(data + layer2_biases)

  # S4 = pooling layer, input of 8x8
  with tf.name_scope("Pool_4"):
    data = tf.nn.avg_pool(data, [1,2,2,1], [1,2,2,1], padding='VALID')

  # F5 = Fully connected layer, input is 4x4
  with tf.name_scope("FCNL_5"):
    size_fc1 = (((image_size - (patch_size-1)) // 2) - (patch_size-1)) // 2
    layer3_weights, layer3_biases = get_weight_and_bias([size_fc1 * size_fc1 * depth, num_hidden])
    shape = data.get_shape().as_list()
    data = tf.reshape(data, [shape[0], shape[1] * shape[2] * shape[3]])
    data = tf.nn.relu(tf.matmul(data, layer3_weights) + layer3_biases)

  with tf.name_scope("FCNL_6"):
    # add dropouts
    data = tf.nn.dropout(data, 0.5)
    layer4_weights, layer4_biases = get_weight_and_bias([num_hidden, num_labels])
    logits = tf.matmul(data, layer4_weights) + layer4_biases
  
  with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)
    with tf.name_scope('total'):
      cross_entropy = tf.reduce_mean(diff)
#   tf.summary.scalar('cross_entropy', cross_entropy)
  
  with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
  
  with tf.name_scope('accuracy'):
    train_prediction = tf.nn.softmax(logits)
    
#   valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
#   test_prediction = tf.nn.softmax(model(tf_test_dataset))
  

In [111]:
log_dir = "/tmp/tensorflow/udacity/"
if tf.gfile.Exists(log_dir):
  tf.gfile.DeleteRecursively(log_dir)
tf.gfile.MakeDirs(log_dir)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(log_dir, graph=graph)

num_steps = 51

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print ("initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0]-batch_size)
    batch_data = train_dataset[offset:(offset+batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, cross_entropy, train_prediction], feed_dict=feed_dict)
    merged = tf.summary.merge_all()
    if (step % (num_steps//10) == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
#       print('Validation accuracy: %.1f%%' % accuracy(
#         valid_prediction.eval(), valid_labels))
#   print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


initialized
Minibatch loss at step 0: 2.429051
Minibatch accuracy: 6.2%
Minibatch loss at step 5: 2.298851
Minibatch accuracy: 12.5%
Minibatch loss at step 10: 2.261606
Minibatch accuracy: 12.5%
Minibatch loss at step 15: 2.185611
Minibatch accuracy: 25.0%
Minibatch loss at step 20: 2.136963
Minibatch accuracy: 31.2%
Minibatch loss at step 25: 2.182964
Minibatch accuracy: 25.0%
Minibatch loss at step 30: 2.145024
Minibatch accuracy: 31.2%
Minibatch loss at step 35: 2.338357
Minibatch accuracy: 12.5%
Minibatch loss at step 40: 1.723234
Minibatch accuracy: 50.0%
Minibatch loss at step 45: 1.728313
Minibatch accuracy: 43.8%
Minibatch loss at step 50: 1.925233
Minibatch accuracy: 31.2%
